![jupyter](./RNN.jpg)

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#load dataset
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [5]:
trainimgs,trainlabels,testimgs,testlabels  = mnist.train.images,mnist.train.labels,mnist.test.images,mnist.test.labels

ntrain,ntest,dim,nclasses = trainimgs.shape[0],testimgs.shape[0],trainimgs.shape[1],trainlabels.shape[1]

# _x(输出)—>_H(128的隐藏层)—>RNN—>LSTM—>输出

In [24]:
diminput = 28
dimhidden = 128
dimoutput = nclasses
#将28*28的图像分割为28个1*28的行像素，拆分为了28个小步   例如【1,2,3,4,5，，，，，，28】像素的单元
nsteps = 28



#第一层为：输入一个28*128的隐藏层
#最后一层为：28*10的输出层
weights = {
    'hidden' : tf.Variable(tf.random_normal([diminput,dimhidden])),
    'out' : tf.Variable(tf.random_normal([dimhidden,dimoutput]))
}

biases = {
    'hidden' : tf.Variable(tf.random_normal([dimhidden])),
    'out' : tf.Variable(tf.random_normal([dimoutput]))
}


In [36]:
def _RNN(_X,_W,_b,_nsteps,_name):
    
    
    #目前的输入_X为batchsize*nsteps*diminput
    #如果要将数据输入，需要进行格式转换，转换为nsteps*batchsize个diminput【28个像素的行】
    _X = tf.transpose(_X,[1,0,2])         #将原输入转化为nsteps*batchsize*diminput
    _X = tf.reshape(_X,[-1,diminput])    #将input reshape为[nsteps*batchsize,diminput]
    
    
    
    
    
    #input layer ==> hidden layer
    _H = tf.matmul(_X,_W['hidden']) + _b['hidden']
    
    #以上得到的输入为nsteps*batchszie(得到的输出相当于将一个batch里面的所有作为一组)
    #但RNN需要的是一个一个的nsteps
    #需要将得到的输出拆分
    _Hsplit = tf.split(0, nsteps, _H)  #把-H按照nsteps进行切分
    
    with tf.Variable_scope(_name) as scope:
        
        
        scope.reuse_variables()
        lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(dimhidden,forget_bias=1.0)
        _LSTM_O,_LSTM_S = tf.nn.rnn(lstm_cell, _Hsplit, dtype=tf.float32)
        
        #output
        _O = tf.matmul(_LSTM_O[-1],_W['out']) + _b['out']
        
        return {
            'X':_X, 'H':_H, 'Hsplit':-Hsplist, 'LSTM_O':_LSTM_O, 'LSTM_S':_LSTM_S, 'O':_O
        }
    
    
    

In [39]:
learning_rate = 0.001
x = tf.placeholder('float32',[None,nsteps,diminput])
y = tf.placeholder('float32',[None,dimoutput])
myrnn = _RNN(x, weights, biases, nsteps, 'basic')
pred = myrnn['O']
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred,y))
optm = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
accr = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(pred,1),tf.argmax(y,1),dtype=tf.float32)))
init = tf.global_variables_initializer()

TypeError: Input 'split_dim' of 'Split' Op has type float32 that does not match expected type of int32.

In [ ]:
training_epoch = 5
batch_size =16
display_step = 1
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(training_epoch):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples/batch_size)
        for i in range(total_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            batch_xs = batch_xs.reshape(batch_size,nsteps,diminput)
            
            feeds = {x:batch_xs,y:batch_ys}
            sess.run(optm,feed_dict=feeds)
            avg_cost += sess.run(cost,feed_dict=feeds).total_batch
            